In [7]:
import pyshark

In [9]:
cap = pyshark.FileCapture('/root/Documents/Thesis/PCAPS/wifi-01.cap', only_summaries=False)

In [20]:
x = 0
num_data_pkts = 10
data_pkts = []
while len(data_pkts) != num_data_pkts:
    
    if (cap[x].highest_layer == "DATA"):
        data_pkts.append(cap[x])
        
    x += 1

In [21]:
len(data_pkts)
data_pkts

[<DATA Packet>,
 <DATA Packet>,
 <DATA Packet>,
 <DATA Packet>,
 <DATA Packet>,
 <DATA Packet>,
 <DATA Packet>,
 <DATA Packet>,
 <DATA Packet>,
 <DATA Packet>]

In [64]:
test_pkt = data_pkts[0]
print test_pkt.wlan.field_names

['', 'fc_frag', 'sa_resolved', 'fc_type_subtype', 'ra', 'seq', 'addr', 'ta', 'fc', 'fc_protected', 'da', 'ccmp_extiv', 'ta_resolved', 'qos_tid', 'fc_moredata', 'staa_resolved', 'fc_subtype', 'flags', 'da_resolved', 'fc_version', 'fc_order', 'qos_priority', 'qos_ack', 'sa', 'duration', 'fc_tods', 'staa', 'wep_key', 'qos', 'addr_resolved', 'fc_ds', 'frag', 'fc_fromds', 'fc_pwrmgt', 'bssid_resolved', 'qos_txop_dur_req', 'qos_amsdupresent', 'fc_retry', 'bssid', 'ra_resolved', 'qos_bit4', 'fc_type']


In [68]:
for pkt in data_pkts:
#     print pkt.frame_info.get_field('encap_type')
    for i, field in enumerate(pkt.wlan.field_names):
        print pkt.wlan.field_names[i] + ": " + pkt.wlan.get_field(field)

: CCMP parameters
fc_frag: 0
sa_resolved: ac:84:c6:97:7c:cc
fc_type_subtype: 40
ra: 78:d2:94:4d:ab:3e
seq: 3224
addr: 78:d2:94:4d:ab:3e
ta: ac:84:c6:97:7c:cc
fc: 0x00008841
fc_protected: 1
da: 78:d2:94:4d:ab:3e
ccmp_extiv: 0x000000176C15
ta_resolved: ac:84:c6:97:7c:cc
qos_tid: 0
fc_moredata: 0
staa_resolved: ac:84:c6:97:7c:cc
fc_subtype: 8
flags: 0x00000041
da_resolved: 78:d2:94:4d:ab:3e
fc_version: 0
fc_order: 0
qos_priority: 0
qos_ack: 0x00000000
sa: ac:84:c6:97:7c:cc
duration: 202
fc_tods: 1
staa: ac:84:c6:97:7c:cc
wep_key: 0
qos: 0x00000000
addr_resolved: 78:d2:94:4d:ab:3e
fc_ds: 0x00000001
frag: 0
fc_fromds: 0
fc_pwrmgt: 0
bssid_resolved: 78:d2:94:4d:ab:3e
qos_txop_dur_req: 0
qos_amsdupresent: 0
fc_retry: 0
bssid: 78:d2:94:4d:ab:3e
ra_resolved: 78:d2:94:4d:ab:3e
qos_bit4: 0
fc_type: 2

: CCMP parameters
fc_frag: 0
sa_resolved: 14:91:82:cd:df:3d
fc_type_subtype: 40
ra: 78:d2:94:4d:ab:3e
seq: 4053
addr: 78:d2:94:4d:ab:3e
ta: 14:91:82:cd:df:3d
fc: 0x00008841
fc_protected: 1
da: 78:d2